# 3D Segmentation with UNet

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/3d_segmentation/unet_segmentation_3d_ignite.ipynb)

## Setup environment

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
 
from glob import glob
import os
import sys
import shutil
import numpy as np

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
#data_dir = "/content/drive/MyDrive/unet-with-monai/dataset/prostate"
data_dir = "./dataset/prostate"
dataset_format_config = np.load("./dataset_format_config.npy", allow_pickle=True).item()
print(dataset_format_config)
expert_ids = np.array([1, 2, 3, 4, 5, 6])

{'training_num': 40, 'validation_num': 6, 'test_num': 9, 'shuffle_id': False, 'directly_mode': True}


## Setup logging

## Setup data
First generate the proper file tree

In [3]:
training_dir = os.path.join(data_dir, 'Training')
validation_dir = os.path.join(data_dir, 'Validation')

new_training_dir = os.path.join(data_dir, 'new_Training')
new_validation_dir = os.path.join(data_dir, 'new_Validation')
new_test_dir = os.path.join(data_dir, 'Test')
os.mkdir(new_training_dir)
os.mkdir(new_validation_dir)
os.mkdir(new_test_dir)
idx = 0

train_num = dataset_format_config['training_num']
val_num = dataset_format_config['validation_num']
test_num = dataset_format_config['test_num']
shuffle = dataset_format_config['shuffle_id']


for case in sorted(os.listdir(training_dir)):
    if shuffle:
        np.random.shuffle(expert_ids)
    print(expert_ids)
    dir = os.path.join(training_dir, case)
    if not os.path.isdir(dir):
        continue

    if idx < train_num:
        class_dir = new_training_dir
    elif idx < train_num+val_num:
        class_dir = new_validation_dir
    else:
        class_dir = new_test_dir

    for file in os.listdir(dir):
        if not file.endswith('.gz'):
            continue
        old_full_name = os.path.join(dir, file)
        for i in range(6):
            if f"seg{i+1:02d}" in file:
                file = file.replace(f"seg{i+1:02d}", f"seg{expert_ids[i]:02d}")
                break
        
        new_full_name = os.path.join(class_dir, f'case{idx:02d}_'+file)
        os.rename(old_full_name, new_full_name)
        print(old_full_name)
        print(new_full_name)
    idx+=1

for case in sorted(os.listdir(validation_dir)):
    if shuffle:
        np.random.shuffle(expert_ids)
    dir = os.path.join(validation_dir, case)
    if not os.path.isdir(dir):
        continue

    if idx < train_num:
        class_dir = new_training_dir
    elif idx < train_num+val_num:
        class_dir = new_validation_dir
    else:
        class_dir = new_test_dir

    for file in os.listdir(dir):
        if not file.endswith('.gz'):
            continue
        old_full_name = os.path.join(dir, file)
        for i in range(6):
            if f"seg{i+1:02d}" in file:
                file = file.replace(f"seg{i+1:02d}", f"seg{expert_ids[i]:02d}")
                break


        new_full_name = os.path.join(class_dir, f'case{idx:02d}_'+file)
        os.rename(old_full_name, new_full_name)
        print(old_full_name)
        print(new_full_name)
    idx+=1



shutil.rmtree(training_dir)
shutil.rmtree(validation_dir)
os.rename(new_training_dir, training_dir)
os.rename(new_validation_dir, validation_dir)

[1 2 3 4 5 6]
./dataset/prostate/Training/case01/image.nii.gz
./dataset/prostate/new_Training/case00_image.nii.gz
./dataset/prostate/Training/case01/task01_seg04.nii.gz
./dataset/prostate/new_Training/case00_task01_seg04.nii.gz
./dataset/prostate/Training/case01/task02_seg03.nii.gz
./dataset/prostate/new_Training/case00_task02_seg03.nii.gz
./dataset/prostate/Training/case01/task01_seg06.nii.gz
./dataset/prostate/new_Training/case00_task01_seg06.nii.gz
./dataset/prostate/Training/case01/task02_seg01.nii.gz
./dataset/prostate/new_Training/case00_task02_seg01.nii.gz
./dataset/prostate/Training/case01/task02_seg05.nii.gz
./dataset/prostate/new_Training/case00_task02_seg05.nii.gz
./dataset/prostate/Training/case01/task01_seg02.nii.gz
./dataset/prostate/new_Training/case00_task01_seg02.nii.gz
./dataset/prostate/Training/case01/task02_seg02.nii.gz
./dataset/prostate/new_Training/case00_task02_seg02.nii.gz
./dataset/prostate/Training/case01/task01_seg05.nii.gz
./dataset/prostate/new_Training/c

./dataset/prostate/Training/case39/task01_seg05.nii.gz
./dataset/prostate/new_Training/case38_task01_seg05.nii.gz
./dataset/prostate/Training/case39/task01_seg03.nii.gz
./dataset/prostate/new_Training/case38_task01_seg03.nii.gz
./dataset/prostate/Training/case39/task02_seg04.nii.gz
./dataset/prostate/new_Training/case38_task02_seg04.nii.gz
./dataset/prostate/Training/case39/task01_seg01.nii.gz
./dataset/prostate/new_Training/case38_task01_seg01.nii.gz
./dataset/prostate/Training/case39/task02_seg06.nii.gz
./dataset/prostate/new_Training/case38_task02_seg06.nii.gz
[1 2 3 4 5 6]
./dataset/prostate/Training/case40/image.nii.gz
./dataset/prostate/new_Training/case39_image.nii.gz
./dataset/prostate/Training/case40/task01_seg04.nii.gz
./dataset/prostate/new_Training/case39_task01_seg04.nii.gz
./dataset/prostate/Training/case40/task02_seg03.nii.gz
./dataset/prostate/new_Training/case39_task02_seg03.nii.gz
./dataset/prostate/Training/case40/task01_seg06.nii.gz
./dataset/prostate/new_Training/c